In [1]:
import pandas as pd
import os
import json
import xlrd

In [2]:
def generateIdDesa(dataDf) :
    return dataDf['Alamat Prov']+dataDf["Alamat Kab"]+dataDf["Alamat Kec"]+dataDf["Alamat Desa"]

In [3]:
# ganti ini ya, sesuai dengan kebutuhan kalian
GLOBAL_VAR = {
    "kode_satker":'1507',
    "master_kecamatan_file":'master data/Master_Kecamatan_1507.xlsx',
}

In [4]:
# data di folder data pendukung
folder = 'data pendukung'
datas = os.listdir(folder)

dfs = []
# buat kode unik harus ada kolom kode_kab, kode_kec, kode_desa, kode_sls, kode_subsls, no urut bangunan, no keluarga setelah verif
for item in datas :
    kegiatan = item
    tempDf = pd.read_excel(folder+'/'+kegiatan, converters={
    'NIK':str, 
    'Alamat Prov':str, 
    'Alamat Kab':str, 
    'Alamat Kec':str, 
    'Alamat Desa':str,
    'Status Seleksi (1=Terpilih, 2=Tidak Terpilih)':str
})
    if 'iddesa' not in tempDf.columns:
        tempDf['iddesa'] = generateIdDesa(tempDf)
    kegiatanPath = '_'+GLOBAL_VAR["kode_satker"]+'_exportmitra.xlsx'
    kegiatan = kegiatan.replace(kegiatanPath, "")
    tempObj = {"kegiatan":kegiatan,
               "mitra":tempDf}
    
    dfs.append(tempObj)

In [5]:
dfs[0]['kegiatan']

'LF_SP2020_1'

In [6]:
# cek data
dfs[0]['mitra'].loc[0:5, ~dfs[0]['mitra'].columns.isin(['NIK', 'Nama', 'Email', 'Username'])]

,Posisi,"Status Seleksi (1=Terpilih, 2=Tidak Terpilih)",Alamat Prov,Alamat Kab,Alamat Kec,Alamat Desa,Alamat Detail,Domisili Sama,Tanggal Lahir (dd/mm/yyyy),NPWP,...,Mengikuti Pendataan BPS,SP,ST,SE,Susenas,Sakernas,SBH,Catatan,Posisi Daftar,iddesa
0,Kortim,1,15,07,021,008,Rt 09 Kuala Dua Dusun Mekarsari,1,26/01/1990,NaN,...,1,1,1,1,1,1,0,NaN,Kortim,1507021008
1,Kortim,1,15,07,021,005,RT 02 kelurahan senyerang,1,16/05/1991,NaN,...,1,1,1,1,0,0,0,NaN,PPL,1507021005
2,PPL,1,15,07,021,002,Jalan Pemuda Rt.06 Desa Sungai Rambai Kecamata...,1,27/08/1990,NaN,...,1,1,0,0,1,1,0,NaN,PPL,1507021002
3,Kortim,1,15,07,031,001,Jl. Lintas Teluk Nilau,1,10/04/1982,73.243.563.1-334.000,...,1,1,1,1,0,0,0,Trim's,PPL,1507031001
4,Kortim,1,15,07,031,010,Lintas roro kuala tungkal,1,05/03/1986,NaN,...,1,1,1,1,1,0,0,Sukses selalu,Kortim,1507031010
5,PPL,1,15,07,031,008,Jl. Simpang Semau RT 03,1,24/03/1989,73.717.000.1-334.000,...,1,1,0,1,0,0,0,"Semoga bisa bekerja sama, terima kasih",PPL,1507031008


In [7]:
# ganti tipe datanya make konverter
konverter = {
               'iddesa':str,
               'kode_prov':str,
               'nama_prov':str,
               'kode_kab':str,
               'nama_kab':str,
               'kode_kec':str,
               'nama_kec':str
            }
master_data_kec =  pd.read_excel(GLOBAL_VAR['master_kecamatan_file'], converters=konverter)
master_data_kec.head()

,kode_prov,nama_prov,kode_kab,nama_kab,kode_kec,nama_kec
0,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU
1,15,JAMBI,07,TANJUNG JABUNG BARAT,011,MERLUNG
2,15,JAMBI,07,TANJUNG JABUNG BARAT,012,BATANG ASAM
3,15,JAMBI,07,TANJUNG JABUNG BARAT,013,TEBING TINGGI
4,15,JAMBI,07,TANJUNG JABUNG BARAT,014,RENAH MENDALUH


In [8]:
kegiatans = []
for item in dfs :
    if item not in kegiatans:
        kegiatans.append(item["kegiatan"])

kegiatans

['LF_SP2020_1',
 'LF_SP2020_3',
 'LF_SP2020_4',
 'REGSOSEK22_1',
 'REGSOSEK22_6',
 'SAKERNAS23-FEB_1',
 'SAKERNAS23-FEB_2',
 'SERUTI23-MAR_1',
 'ST2023-PW_1',
 'ST2023-PW_2',
 'SUSENAS23-MAR_1',
 'UBINAN23-SUB1_1',
 'UDP23_1']

In [9]:
# data yang dihasilkan di output file nanti, silahkan tambahkan atau kurangin
kolomOutput = [
    "NIK",
    "Nama",
    "Username",
    "No Telp",
    "Alamat Prov",
    "Alamat Kab",
    "Alamat Kec",
    "Kegiatan Diikuti"
    ]

usernames = []
mitraFinal = []

for dataKegiatan in dfs:
    mitraKegiatan = dataKegiatan['mitra']
    kegiatan = dataKegiatan['kegiatan']
    for index, row in mitraKegiatan.iterrows():

        if row['Username'] not in usernames:
            usernames.append(row['Username'])
            mitra = {}
            for kolom in kolomOutput:
                match kolom :
                    case 'Kegiatan Diikuti':
                        # inisiasi nilai karena sebelumnya belum ada username di
                        # cuma masukin kegiatan yang diikuti kalau dia lulus
                        if row['Status Seleksi (1=Terpilih, 2=Tidak Terpilih)'] == '1':
                            mitra[kolom] = [kegiatan]
                        else:
                            mitra[kolom] = []
                    case 'Alamat Prov':
                        # mitra[kolom]=desa['nmprov']
                        idx = master_data_kec.loc[master_data_kec['kode_prov']==row[kolom], "nama_prov"].index
                        if (len(idx)<1):
                            mitra[kolom]='Data Tidak Tersedia'
                        else :
                             mitra[kolom] = master_data_kec.loc[idx[0], 'nama_prov']

                    case 'Alamat Kab':
                        # mitra[kolom]=desa['nmkab']
                        idx = master_data_kec.loc[master_data_kec['kode_kab']==row[kolom], "nama_kab"].index
                        if (len(idx)<1):
                            mitra[kolom]='Data Tidak Tersedia'
                        else :
                             mitra[kolom] = master_data_kec.loc[idx[0], 'nama_kab']
                    case 'Alamat Kec':
                        idx = master_data_kec.loc[master_data_kec['kode_kec']==row[kolom], "nama_kec"].index
                        if (len(idx)<1):
                            mitra[kolom]='Data Tidak Tersedia'
                        else :
                             mitra[kolom] = master_data_kec.loc[idx[0], 'nama_kec']
                        # mitra[kolom]=desa['nmkec']
                    case _:
                        mitra[kolom] = row[kolom]

            for kegiatanx in kegiatans:
                mitra[kegiatanx] = 'Tidak Mendaftar'
            if row['Status Seleksi (1=Terpilih, 2=Tidak Terpilih)'] =='1':
                mitra[kegiatan] = 'Lulus'
            elif row['Status Seleksi (1=Terpilih, 2=Tidak Terpilih)'] =='2':
                mitra[kegiatan] = 'Tidak Lulus'

            mitraFinal.append(mitra)
        else :

            index = usernames.index(row['Username'])
            if row['Status Seleksi (1=Terpilih, 2=Tidak Terpilih)'] =='1':
                mitraFinal[index]['Kegiatan Diikuti'].append(kegiatan)
                mitraFinal[index][kegiatan] = 'Lulus'
            elif row['Status Seleksi (1=Terpilih, 2=Tidak Terpilih)'] =='2':
                mitraFinal[index][kegiatan] = 'Tidak Lulus'


            


    

In [19]:
mitraFinal[0]

{'NIK': '1506132601900001',
 'Nama': 'Joni wahyudi',
 'Username': 'Joniwahyudi',
 'No Telp': '+62 852-6689-2830',
 'Alamat Prov': 'JAMBI',
 'Alamat Kab': 'TANJUNG JABUNG BARAT',
 'Alamat Kec': 'SENYERANG',
 'Kegiatan Diikuti': ['LF_SP2020_1',
  'REGSOSEK22_1',
  'ST2023-PW_1',
  'SUSENAS23-MAR_1'],
 'LF_SP2020_1': 'Lulus',
 'LF_SP2020_3': 'Tidak Mendaftar',
 'LF_SP2020_4': 'Tidak Mendaftar',
 'REGSOSEK22_1': 'Lulus',
 'REGSOSEK22_6': 'Tidak Mendaftar',
 'SAKERNAS23-FEB_1': 'Tidak Mendaftar',
 'SAKERNAS23-FEB_2': 'Tidak Mendaftar',
 'SERUTI23-MAR_1': 'Tidak Mendaftar',
 'ST2023-PW_1': 'Lulus',
 'ST2023-PW_2': 'Tidak Mendaftar',
 'SUSENAS23-MAR_1': 'Lulus',
 'UBINAN23-SUB1_1': 'Tidak Mendaftar',
 'UDP23_1': 'Tidak Mendaftar'}

In [20]:
# tambah kolom total kegiatan yang diikutin 
# dan transformasi kolom kegiatan yang diikuti
# menggunakan array join

for mitra in mitraFinal :
    mitra['Jumlah Kegiatan Diikuti'] = len(mitra['Kegiatan Diikuti'])
    mitra["Kegiatan Diikuti"] = ','.join(mitra["Kegiatan Diikuti"])

# filter hanya yang mengikuti kegiatan
mitraFinalMinimal1Kegiatan = [x for x in mitraFinal if x['Jumlah Kegiatan Diikuti']!=0]
    

In [21]:
mitraFinalMinimal1Kegiatan[0]

{'NIK': '1506132601900001',
 'Nama': 'Joni wahyudi',
 'Username': 'Joniwahyudi',
 'No Telp': '+62 852-6689-2830',
 'Alamat Prov': 'JAMBI',
 'Alamat Kab': 'TANJUNG JABUNG BARAT',
 'Alamat Kec': 'SENYERANG',
 'Kegiatan Diikuti': 'LF_SP2020_1,REGSOSEK22_1,ST2023-PW_1,SUSENAS23-MAR_1',
 'LF_SP2020_1': 'Lulus',
 'LF_SP2020_3': 'Tidak Mendaftar',
 'LF_SP2020_4': 'Tidak Mendaftar',
 'REGSOSEK22_1': 'Lulus',
 'REGSOSEK22_6': 'Tidak Mendaftar',
 'SAKERNAS23-FEB_1': 'Tidak Mendaftar',
 'SAKERNAS23-FEB_2': 'Tidak Mendaftar',
 'SERUTI23-MAR_1': 'Tidak Mendaftar',
 'ST2023-PW_1': 'Lulus',
 'ST2023-PW_2': 'Tidak Mendaftar',
 'SUSENAS23-MAR_1': 'Lulus',
 'UBINAN23-SUB1_1': 'Tidak Mendaftar',
 'UDP23_1': 'Tidak Mendaftar',
 'Jumlah Kegiatan Diikuti': 4}

In [22]:
jsonObj = json.dumps(mitraFinalMinimal1Kegiatan)
mitraFinalPd = pd.read_json(jsonObj, dtype={
    "NIK":str
})
outputPath = 'hasil/'+GLOBAL_VAR['kode_satker']+'.csv'
mitraFinalPd.to_csv(outputPath)
mitraFinalPd.loc[0:5, ~mitraFinalPd.columns.isin(['NIK', 'Nama', 'Email', 'Username'])]

,No Telp,Alamat Prov,Alamat Kab,Alamat Kec,Kegiatan Diikuti,LF_SP2020_1,LF_SP2020_3,LF_SP2020_4,REGSOSEK22_1,REGSOSEK22_6,SAKERNAS23-FEB_1,SAKERNAS23-FEB_2,SERUTI23-MAR_1,ST2023-PW_1,ST2023-PW_2,SUSENAS23-MAR_1,UBINAN23-SUB1_1,UDP23_1,Jumlah Kegiatan Diikuti
0,+62 852-6689-2830,JAMBI,TANJUNG JABUNG BARAT,SENYERANG,"LF_SP2020_1,REGSOSEK22_1,ST2023-PW_1,SUSENAS23...",Lulus,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,4
1,+62 822-8182-5948,JAMBI,TANJUNG JABUNG BARAT,SENYERANG,"LF_SP2020_1,REGSOSEK22_1,ST2023-PW_1,UBINAN23-...",Lulus,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,4
2,+62 823-7185-0760,JAMBI,TANJUNG JABUNG BARAT,SENYERANG,"LF_SP2020_1,REGSOSEK22_1,SERUTI23-MAR_1,SUSENA...",Lulus,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,4
3,+62 081-2744-88254,JAMBI,TANJUNG JABUNG BARAT,BRAM ITAM,"LF_SP2020_1,REGSOSEK22_1,ST2023-PW_1,SUSENAS23...",Lulus,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,4
4,+62 852-6608-5934,JAMBI,TANJUNG JABUNG BARAT,BRAM ITAM,"LF_SP2020_1,REGSOSEK22_1,ST2023-PW_1",Lulus,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,3
5,+62 856-5864-0674,JAMBI,TANJUNG JABUNG BARAT,BRAM ITAM,"LF_SP2020_1,REGSOSEK22_1,ST2023-PW_1",Lulus,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Lulus,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,Tidak Mendaftar,3
